In [60]:
import requests
import json
import time
import datetime

In [109]:
#Credentials
USERNAME="sarang0909.bds@gmail.com"
PASSWORD="Fourthyear@4u"

In [110]:
#Varaibles
repo_data=[]
repo_id=[]
count_open_issue=[]
count_forks=[]
count_watchers=[]
PULL_DATA=[]
Pull_json_data=[]  
Final_json_data=[]
contri_count = []
push_delta=[]
contributor_list=[]
prob_pull=[]
prob_contributors=[]
temp=[]
currenttime = time.strftime("%Y-%m-%dT%H:%M:%SZ",time.localtime())

In [111]:
repo_url = 'https://api.github.com/repos/scikit-learn/scikit-learn'

In [112]:
def repos_data(repo_url):
        main_data=requests.get(repo_url,auth=(USERNAME,PASSWORD))
        if main_data.status_code == 200:
            repo_data.append(main_data.json())
            #Repo id extraction for our reference
            repo_id.append(main_data.json()['id'])
            #Get the contributors list
            con_url= repo_url + "/contributors"
            actual_count=[]
            count_URL=requests.get(url=con_url,auth=(USERNAME,PASSWORD))
            if count_URL.status_code == 204:
                contributor_list.append('NA')
                contri_count.append(0)
            else:    
                count_con=count_URL.json()
                if type(count_con) is list:
                    for j in range(len(count_con)):
                        if 'login' in count_con[j]:    
                            actual_count.append(count_con[j]['login'])
                        else:
                            actual_count.append('NA')
                    contributor_list.append(actual_count)    
                    contri_count.append(len(actual_count))
                else:
                    contributor_list.append('NA')
                    contri_count.append(0)

In [113]:
def cal_total_page(header_data):
    if 'Link' in header_data:
        print(header_data['Link'])
        target_data=header_data['Link'].split(",")
        id_no=target_data[1].find("&")
        total_page_val=target_data[1][id_no-2:id_no]
        if '=' in total_page_val:
            page_value=total_page_val.replace("=",'')
            total_page=int(page_value)
            return total_page
        else:
            total_page=int(total_page_val)
            return total_page
    else:
        return 0

In [114]:
#Function for features at repo level
def fatures_at_repo_count(repo_data):
    for i in range(len(repo_data)):
        #count for open issue per repository    
        count_open_issue.append(repo_data[i]['open_issues_count'])    
        #count for number of forks
        count_forks.append(repo_data[i]['forks_count'])
        #count for watchers per repo
        count_watchers.append(repo_data[i]['watchers_count'])
        #Tme since last push on repository    
        a = datetime.datetime.strptime(repo_data[i]['pushed_at'], "%Y-%m-%dT%H:%M:%SZ")
        b = datetime.datetime.strptime(currenttime, "%Y-%m-%dT%H:%M:%SZ")
        serconds_last_push = (b-a).total_seconds()
        push_delta.append(serconds_last_push)
        #PUll Data
        Pull_URL=repo_data[i]['pulls_url']
        PULL_HEADER_URL=Pull_URL.replace('{/number}','?page=1&per_page=100&state=all')
        cal_page= requests.get(url=PULL_HEADER_URL,auth=(USERNAME,PASSWORD))
        header_data=cal_page.headers
        total_page=cal_total_page(header_data)
        print('total pages:{}'.format(total_page))
        PULL_raw_data=[]
        for j in range(total_page):
            #print("Processing total pull data inside one repo...")
            target_str='?page=%d&per_page=100&state=all'%j
            PULL_URL = Pull_URL.replace('{/number}', target_str)
            PULL_raw_data.append(requests.get(url=PULL_URL,auth=(USERNAME,PASSWORD)).json())
        print(len(PULL_raw_data))
        for k in range(len(PULL_raw_data)):
            print("Pull raw data count %d"%k)
            print(len(PULL_raw_data[k]))
            for m in range(len(PULL_raw_data[k])):
                
                temp.append(PULL_raw_data[k][m])
        PULL_DATA.append(temp)   
        #PULL_URL=Pull_URL.replace('{/number}','?state=all')
        #PULL_DATA.append(requests.get(url=PULL_URL,auth=(USERNAME,PASSWORD)).json())
        

In [115]:
#calculating probability for contri count    
def prob_count_contributors(PULL_DATA):
    print("count prob %de"%len(PULL_DATA))
    for i in range(len(PULL_DATA)):
        print("count %d"%i)
        if ' ' not in PULL_DATA[i]:
            total_merged = 0
            contributors_merged= 0
            pulls_closed = 0
            contributors_prob = 0
            pull_prob = 0
            for j in range(len(PULL_DATA[i])):
                if (PULL_DATA[i][j]['state'] == 'closed'):
                     pulls_closed+=1
                if(PULL_DATA[i][j]['merged_at']!=None):                
                     total_merged+=1     
                if(PULL_DATA[i][j]['merged_at']!=None and PULL_DATA[i][j]['user']['login'] in contributor_list[i] ):
                     contributors_merged+=1       
            if(total_merged!=0):
                contributors_prob = contributors_merged/total_merged 
                prob_contributors.append(contributors_prob)
            else:
                prob_contributors.append(0)
            if(pulls_closed!=0):
                pull_prob = total_merged/pulls_closed 
                prob_pull.append(pull_prob)
            else:
                prob_pull.append(0)
        else:
           
            prob_contributors.append(0)
            prob_pull.append(0)

In [116]:
#PULL_DATA contains data of all pull request regarding to that particular repo
#label 1 :- if pull request is merge and considered as accepted.
#label 0:- pull request is not merge and state is closed so rejected
#label -1:- pull request is not merge and state is open so open                    
def label(PULL_DATA):
    #Get the correct label URL
    label_url = PULL_DATA['url']+"/merge"
    #requests.get(uel,auth) gives only the response. but requests.get(uel,auth).json gives json data of resonse 
    label_data = requests.get(url=label_url,auth=(USERNAME,PASSWORD))
    label_status = label_data.status_code
    #condition for accepte. 204 is http status code for no content found
    if label_status == 204:
        return 1
    #404 not found so gets rejected on the basis of state value
    if label_status == 404:
        if PULL_DATA['state'] == 'closed':#rejected
            return 0
        elif PULL_DATA['state'] == 'open':#open
            return -1
    else:
        return 2

In [117]:
#function for calculate no_of file change and nos of additions and deletions    
def diff_status(PULL_URL):
    PULL_URL=PULL_URL+"/files"
    PATCH_DATA = requests.get(url=PULL_URL,auth=(USERNAME,PASSWORD)).json()
    DATA=[]
    insertions=0
    deletions=0
    #checking len of data to ensure that correct data will return
    if len(PATCH_DATA)>0:
        for i in range(len(PATCH_DATA)):
            #checking condition for data that is not releveant to us and in every such instance that 
            #data contains message key. verified this key many times.
            if 'message' in PATCH_DATA:
                DATA.append(0)
                DATA.append(insertions)
                DATA.append(deletions)
                return DATA
            else:
                #calculating additions and insertions
                insertions=insertions+PATCH_DATA[i]['additions']
                insertions=insertions+0
                deletions=deletions+PATCH_DATA[i]['deletions']
                deletions=deletions+0
        DATA.append(len(PATCH_DATA))
        DATA.append(insertions)
        DATA.append(deletions)  
        return DATA
    else:
        DATA.append(0)
        DATA.append(insertions)
        DATA.append(deletions)
        return DATA

In [118]:
#function for commits calcularion    
def commits_count(URL):
    count=requests.get(url=URL,auth=(USERNAME,PASSWORD)).json()
    return count

In [122]:
#Gathering data at PUll level
def PULL_LEVEL_DATA(PULL_DATA):
    
    print("Length of pull data to be collected from  repos:{}".format(len(PULL_DATA)))
    for i in range(len(PULL_DATA)):
        print("Pull level data collection Processing....")
    #if no objectes are there
        if ' ' not in PULL_DATA[i]:
            
            file_json_data=[]
            for j in range(50):
            #for j in range(len(PULL_DATA[i])):
                pull_id=PULL_DATA[i][j]['id']#Pull id for our ref
                no_of_file_changed=0
                no_of_insertion=0
                no_of_deletion=0
                commit_count=0
                acc_label="NA" #for our ref
                print(PULL_DATA[i][j])
                if 'url' in PULL_DATA[i][j]:
                    PULL_URL=PULL_DATA[i][j]['url']
                    #status of pull request
                    acc_label=label(PULL_DATA[i][j])
                    #file level data
                    DATA=diff_status(PULL_URL)
                    #data1=DATA[0].strip().split(" ")
                    no_of_file_changed=DATA[0]
                    no_of_insertion=DATA[1]
                    no_of_deletion=DATA[2]
                if 'commits_url' in PULL_DATA[i][j]:
                    URL=PULL_DATA[i][j]['commits_url']
                    commit_data=commits_count(URL)
                    commit_count=len(commit_data)   
                file_json_data.append({
                   'Pull_id' : pull_id,
                   'label' : acc_label,
                   'no_of_file_change' : no_of_file_changed,
                   'no_of_insertion' : no_of_insertion,
                   'no_of_deletion' : no_of_deletion,
                   'no_of_commits' : commit_count
                    }) 
            
        else:
            Pull_json_data.append([{
                   'Pull_id' : 0,
                   'label' : 0,
                   'no_of_file_change' : 0,
                   'no_of_total_line_changed' : 0,
                   'no_of_commits' : 0
                   }])
            
        Pull_json_data.append(file_json_data)

In [120]:
#Creating Final dictionary
def creating_json_data(length):
    final_data=[]
    for i in range(len(count_forks)):
        final_data.append({
        'id' : repo_id[i],
        'contributors_count' : contri_count[i],
        'open_issue_count' : count_open_issue[i],
        'forks_count' : count_forks[i],
        'watchers_count' : count_watchers[i],
        'Pushed_delta' : push_delta[i],
        'contrib_prob' : prob_contributors[i],
        'repo_prob' : prob_pull[i],
        'pull_data' : Pull_json_data[i]

        })
    return final_data

In [121]:
#travesring and collcting some features at repo level    
repos_data(repo_url)  
print("Successfully collected repos data.\n")  
#More features at repo level
fatures_at_repo_count(repo_data)
print("Successfully collected repos count data.\n")
prob_count_contributors(PULL_DATA) 
print("Successfully collected repo contri count data.\n")
#Features at Pull level
PULL_LEVEL_DATA(PULL_DATA) 
print("Successfully collected pull level data.\n")
#final json data      
Final_json_data=creating_json_data(len(count_forks))
print("Successfully created json data.\n")
#dumping of all data into one json file                                                                                                                                            
with open('Extracted_features.json', 'w') as outfile:  
   json.dump(Final_json_data, outfile)   
print("Succesfull completed data extraction.") 
outfile.close() 

Successfully collected repos data.

<https://api.github.com/repositories/843222/pulls?page=2&per_page=100&state=all>; rel="next", <https://api.github.com/repositories/843222/pulls?page=63&per_page=100&state=all>; rel="last"
total pages:63
63
Pull raw data count 0
100
Pull raw data count 1
100
Pull raw data count 2
100
Pull raw data count 3
100
Pull raw data count 4
100
Pull raw data count 5
100
Pull raw data count 6
100
Pull raw data count 7
100
Pull raw data count 8
100
Pull raw data count 9
100
Pull raw data count 10
100
Pull raw data count 11
100
Pull raw data count 12
100
Pull raw data count 13
100
Pull raw data count 14
100
Pull raw data count 15
100
Pull raw data count 16
100
Pull raw data count 17
100
Pull raw data count 18
100
Pull raw data count 19
100
Pull raw data count 20
100
Pull raw data count 21
100
Pull raw data count 22
100
Pull raw data count 23
100
Pull raw data count 24
100
Pull raw data count 25
100
Pull raw data count 26
100
Pull raw data count 27
100
Pull raw dat

NameError: name 'org_data' is not defined

In [123]:
PULL_LEVEL_DATA(PULL_DATA) 

Length of pull data to be collected from  repos:1
Pull level data collection Processing....
{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11424', 'id': 199131348, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk5MTMxMzQ4', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11424', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11424.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11424.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11424', 'number': 11424, 'state': 'open', 'locked': False, 'title': 'replace log with log1p', 'user': {'login': 'meetnaren', 'id': 6798761, 'node_id': 'MDQ6VXNlcjY3OTg3NjE=', 'avatar_url': 'https://avatars0.githubusercontent.com/u/6798761?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/meetnaren', 'html_url': 'https://github.com/meetnaren', 'followers_url': 'https://api.github.com/users/meetnaren/followers', 'following_url': 'https://api.github.co

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11422', 'id': 199107531, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk5MTA3NTMx', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11422', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11422.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11422.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11422', 'number': 11422, 'state': 'open', 'locked': False, 'title': 'MNT Release 0.19.2 with python 3.7 support', 'user': {'login': 'jnothman', 'id': 78827, 'node_id': 'MDQ6VXNlcjc4ODI3', 'avatar_url': 'https://avatars2.githubusercontent.com/u/78827?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jnothman', 'html_url': 'https://github.com/jnothman', 'followers_url': 'https://api.github.com/users/jnothman/followers', 'following_url': 'https://api.github.com/users/jnothman/following{/other_user}', 'gists_url': 'https://api.github.com/users

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11421', 'id': 199102758, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk5MTAyNzU4', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11421', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11421.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11421.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11421', 'number': 11421, 'state': 'open', 'locked': False, 'title': '[WIP]\xa0Added docstring checks for feature_selection, kernel_approximation and neighbors modules', 'user': {'login': 'NicolasHug', 'id': 1190450, 'node_id': 'MDQ6VXNlcjExOTA0NTA=', 'avatar_url': 'https://avatars2.githubusercontent.com/u/1190450?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/NicolasHug', 'html_url': 'https://github.com/NicolasHug', 'followers_url': 'https://api.github.com/users/NicolasHug/followers', 'following_url': 'https://api.github.com/users/Nicol

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11420', 'id': 199098731, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk5MDk4NzMx', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11420', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11420.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11420.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11420', 'number': 11420, 'state': 'open', 'locked': False, 'title': '[MRG] Use GEMM in _update_dict', 'user': {'login': 'jakirkham', 'id': 3019665, 'node_id': 'MDQ6VXNlcjMwMTk2NjU=', 'avatar_url': 'https://avatars1.githubusercontent.com/u/3019665?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jakirkham', 'html_url': 'https://github.com/jakirkham', 'followers_url': 'https://api.github.com/users/jakirkham/followers', 'following_url': 'https://api.github.com/users/jakirkham/following{/other_user}', 'gists_url': 'https://api.github.com/user

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11419', 'id': 199093093, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk5MDkzMDkz', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11419', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11419.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11419.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11419', 'number': 11419, 'state': 'open', 'locked': False, 'title': '[MRG] Openml data loader', 'user': {'login': 'janvanrijn', 'id': 2292730, 'node_id': 'MDQ6VXNlcjIyOTI3MzA=', 'avatar_url': 'https://avatars1.githubusercontent.com/u/2292730?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/janvanrijn', 'html_url': 'https://github.com/janvanrijn', 'followers_url': 'https://api.github.com/users/janvanrijn/followers', 'following_url': 'https://api.github.com/users/janvanrijn/following{/other_user}', 'gists_url': 'https://api.github.com/users

...

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11335', 'id': 196337248, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk2MzM3MjQ4', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11335', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11335.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11335.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11335', 'number': 11335, 'state': 'closed', 'locked': False, 'title': '[MRG] DOC encourage contributors to look at related projects too', 'user': {'login': 'jnothman', 'id': 78827, 'node_id': 'MDQ6VXNlcjc4ODI3', 'avatar_url': 'https://avatars2.githubusercontent.com/u/78827?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jnothman', 'html_url': 'https://github.com/jnothman', 'followers_url': 'https://api.github.com/users/jnothman/followers', 'following_url': 'https://api.github.com/users/jnothman/following{/other_user}', 'gists_url': 'http

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11334', 'id': 196325257, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk2MzI1MjU3', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11334', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11334.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11334.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11334', 'number': 11334, 'state': 'closed', 'locked': False, 'title': 'DOC: Clarified different conf-mat axes in Wikipedia example #11326', 'user': {'login': 'maxcopeland', 'id': 28763901, 'node_id': 'MDQ6VXNlcjI4NzYzOTAx', 'avatar_url': 'https://avatars1.githubusercontent.com/u/28763901?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/maxcopeland', 'html_url': 'https://github.com/maxcopeland', 'followers_url': 'https://api.github.com/users/maxcopeland/followers', 'following_url': 'https://api.github.com/users/maxcopeland/following{/other

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11333', 'id': 196324239, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk2MzI0MjM5', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11333', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11333.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11333.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11333', 'number': 11333, 'state': 'open', 'locked': False, 'title': 'BUG: Fixes _BaseCompostion._set_params broken where there are no estimators', 'user': {'login': 'thomasjpfan', 'id': 5402633, 'node_id': 'MDQ6VXNlcjU0MDI2MzM=', 'avatar_url': 'https://avatars2.githubusercontent.com/u/5402633?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/thomasjpfan', 'html_url': 'https://github.com/thomasjpfan', 'followers_url': 'https://api.github.com/users/thomasjpfan/followers', 'following_url': 'https://api.github.com/users/thomasjpfan/following{/

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11328', 'id': 196132897, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk2MTMyODk3', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11328', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11328.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11328.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11328', 'number': 11328, 'state': 'closed', 'locked': False, 'title': '[MRG+1]  Doc: complete PR #11180 for the reorganization of the dataset loading utilities section', 'user': {'login': 'jeremiedbb', 'id': 34657725, 'node_id': 'MDQ6VXNlcjM0NjU3NzI1', 'avatar_url': 'https://avatars2.githubusercontent.com/u/34657725?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jeremiedbb', 'html_url': 'https://github.com/jeremiedbb', 'followers_url': 'https://api.github.com/users/jeremiedbb/followers', 'following_url': 'https://api.github.com/users/je

{'url': 'https://api.github.com/repos/scikit-learn/scikit-learn/pulls/11327', 'id': 196097141, 'node_id': 'MDExOlB1bGxSZXF1ZXN0MTk2MDk3MTQx', 'html_url': 'https://github.com/scikit-learn/scikit-learn/pull/11327', 'diff_url': 'https://github.com/scikit-learn/scikit-learn/pull/11327.diff', 'patch_url': 'https://github.com/scikit-learn/scikit-learn/pull/11327.patch', 'issue_url': 'https://api.github.com/repos/scikit-learn/scikit-learn/issues/11327', 'number': 11327, 'state': 'closed', 'locked': False, 'title': '[MRG] Large sparse matrix support', 'user': {'login': 'jnothman', 'id': 78827, 'node_id': 'MDQ6VXNlcjc4ODI3', 'avatar_url': 'https://avatars2.githubusercontent.com/u/78827?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jnothman', 'html_url': 'https://github.com/jnothman', 'followers_url': 'https://api.github.com/users/jnothman/followers', 'following_url': 'https://api.github.com/users/jnothman/following{/other_user}', 'gists_url': 'https://api.github.com/users/jnothm